In [1]:
import torchaudio
import torch
import pandas as pd
from speechbrain.pretrained import EncoderClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

!export OMP_NUM_THREADS=1

/home/lorenzg/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from datasets import load_dataset

# To load common voice
common_voice_version = "mozilla-foundation/common_voice_11_0"
language = "de"
common_voice = load_dataset(common_voice_version, language, split="validation", streaming=True)

/data/vokquant/accent_recog/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [8]:
cv_filtered = common_voice.filter(lambda example: len(example['accent']) > 0)
#print(next(iter(common_voice)))

In [9]:
print(next(iter(cv_filtered)))

Reading metadata...: 16082it [00:02, 7302.03it/s]


{'client_id': '5dfa978d363da129606377dc0be9b4027132c57b31543b9a1b1dfbacb2e22fc85fc8e052de828a06cba179ec26a2fd36d3f2a2047316597af0c4f8d1a69191f5', 'path': 'de_dev_0/common_voice_de_17745113.mp3', 'audio': {'path': 'de_dev_0/common_voice_de_17745113.mp3', 'array': array([ 0.00000000e+00,  5.34275730e-15,  3.55570286e-15, ...,
       -3.36459547e-04, -2.59274646e-04,  6.84560800e-05]), 'sampling_rate': 48000}, 'sentence': 'Das ist doch absolut lächerlich!', 'up_votes': 2, 'down_votes': 0, 'age': 'thirties', 'gender': 'male', 'accent': 'Deutschland Deutsch', 'locale': 'de', 'segment': ''}


In [10]:
hola = cv_filtered.take(10)

In [11]:
list(hola)

Reading metadata...: 16082it [00:01, 11616.95it/s]


[{'client_id': '5dfa978d363da129606377dc0be9b4027132c57b31543b9a1b1dfbacb2e22fc85fc8e052de828a06cba179ec26a2fd36d3f2a2047316597af0c4f8d1a69191f5',
  'path': 'de_dev_0/common_voice_de_17745113.mp3',
  'audio': {'path': 'de_dev_0/common_voice_de_17745113.mp3',
   'array': array([ 0.00000000e+00,  5.34275730e-15,  3.55570286e-15, ...,
          -3.36459547e-04, -2.59274646e-04,  6.84560800e-05]),
   'sampling_rate': 48000},
  'sentence': 'Das ist doch absolut lächerlich!',
  'up_votes': 2,
  'down_votes': 0,
  'age': 'thirties',
  'gender': 'male',
  'accent': 'Deutschland Deutsch',
  'locale': 'de',
  'segment': ''},
 {'client_id': '5dfa978d363da129606377dc0be9b4027132c57b31543b9a1b1dfbacb2e22fc85fc8e052de828a06cba179ec26a2fd36d3f2a2047316597af0c4f8d1a69191f5',
  'path': 'de_dev_0/common_voice_de_17745114.mp3',
  'audio': {'path': 'de_dev_0/common_voice_de_17745114.mp3',
   'array': array([ 0.00000000e+00,  2.36197424e-15, -2.84470700e-15, ...,
           0.00000000e+00,  0.00000000e+00,